In [2]:
import pandas as pd
import pycountry

#### Working with data set ds.salaries.csv, with goal of converting ISO 3166 country code to country name column

In [3]:
df = pd.read_csv("ds_salaries.csv", index_col="#")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
#,,,,,,,,,,,
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


#### Testing conversion with pycountry

In [4]:
df.loc[1]["company_location"]
print(pycountry.countries.get(alpha_2=df.loc[1]["company_location"]).name)

Japan


#### Issue: Certain country codes causing LookupError.  New goal to identify erronous country code.

In [12]:
df.sort_values("company_location", inplace=True)
# print(df["company_location"].unique())
for country in df["company_location"].unique():
    try:

        print(f"{country} country name is {pycountry.countries.get(alpha_2=country).name}")
    except LookupError:
        print(f"{country} country name is AN INVALID FORMAT")


AE country name is United Arab Emirates
AS country name is American Samoa
AT country name is Austria
AU country name is Australia
BE country name is Belgium
BR country name is Brazil
CA country name is Canada
CH country name is Switzerland
CL country name is Chile
CN country name is China
CO country name is Colombia
CZ country name is Czechia
DE country name is Germany
DK country name is Denmark
DZ country name is Algeria
EE country name is Estonia
ES country name is Spain
FR country name is France
GB country name is United Kingdom
GR country name is Greece
HN country name is Honduras
HR country name is Croatia
HU country name is Hungary
IE country name is Ireland
IL country name is Israel
IN country name is India
IQ country name is Iraq
IR country name is Iran, Islamic Republic of
IT country name is Italy
JP country name is Japan
KE country name is Kenya
LU country name is Luxembourg
MD country name is Moldova, Republic of
MT country name is Malta
MX country name is Mexico
MY country 

#### Below attempt is still throwing lookup error - switching to using extra .csv file of countries

In [18]:
try:
    df["company_location"] = (pycountry.countries.lookup(df["company_location"]).name)
    df.head()
except LookupError:
    print("Lookup Error")

Lookup Error


#### Importing of country name/code csv, and removal of unnecessary columns (preparation for merging DF's)

In [27]:
country_df = pd.read_csv("wikipedia-iso-country-codes.csv", index_col="Alpha-2")
country_df.drop(["Alpha-3", "numeric_code", "ISO_3166-2"], axis=1, inplace=True)
country_df.head(70)


,country_name
Alpha-2,
AF,Afghanistan
AX,Åland Islands
AL,Albania
DZ,Algeria
AS,American Samoa
...,...
GQ,Equatorial Guinea
ER,Eritrea
EE,Estonia


#### Merging of DF's based on "company_location" country code

In [36]:
#print(country_df.loc[df.loc[1]["company_location"]]["country_name"])
merged_df = pd.merge(df, country_df, left_on="company_location", right_on="Alpha-2")
print(merged_df["country_name"].unique())


['United Arab Emirates' 'American Samoa' 'Austria' 'Australia' 'Belgium'
 'Brazil' 'Canada' 'Switzerland' 'Chile' 'China' 'Colombia'
 'Czech Republic' 'Germany' 'Denmark' 'Algeria' 'Estonia' 'Spain' 'France'
 'United Kingdom' 'Greece' 'Honduras' 'Croatia' 'Hungary' 'Ireland'
 'Israel' 'India' 'Iraq' 'Iran, Islamic Republic of' 'Italy' 'Japan'
 'Kenya' 'Luxembourg' 'Moldova, Republic of' 'Malta' 'Mexico' 'Malaysia'
 'Nigeria' 'Netherlands' 'New Zealand' 'Pakistan' 'Poland' 'Portugal'
 'Romania' 'Russian Federation' 'Singapore' 'Slovenia' 'Turkey' 'Ukraine'
 'United States' 'Viet Nam']


#### Cleanup of columns including:
* Dropping company_location column
* Renaming country_name to company_location

In [37]:
merged_df_two = merged_df.drop("company_location", axis=1).rename(columns={"country_name": "company_location"})
merged_df_two.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_size,company_location
0,2020,MI,FT,Lead Data Scientist,115000,USD,115000,AE,0,L,United Arab Emirates
1,2022,SE,FT,Machine Learning Engineer,65000,USD,65000,AE,100,S,United Arab Emirates
2,2022,SE,FT,Machine Learning Engineer,120000,USD,120000,AE,100,S,United Arab Emirates
3,2021,EN,FT,AI Scientist,1335000,INR,18053,IN,100,S,American Samoa
4,2020,MI,FT,Data Engineer,65000,EUR,74130,AT,50,L,Austria


#### Rinse & repeat with employee_residence column

In [39]:
employee_merged_df = pd.merge(merged_df_two, country_df, left_on="employee_residence", right_on="Alpha-2")
employee_merged_df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_size,company_location,country_name
0,2020,MI,FT,Lead Data Scientist,115000,USD,115000,AE,0,L,United Arab Emirates,United Arab Emirates
1,2022,SE,FT,Machine Learning Engineer,65000,USD,65000,AE,100,S,United Arab Emirates,United Arab Emirates
2,2022,SE,FT,Machine Learning Engineer,120000,USD,120000,AE,100,S,United Arab Emirates,United Arab Emirates
3,2021,EN,FT,AI Scientist,1335000,INR,18053,IN,100,S,American Samoa,India
4,2021,EN,FT,Big Data Engineer,435000,INR,5882,IN,0,L,Switzerland,India


In [ ]:
final_df = merged_df.drop("employee_residence", axis=1).rename(columns={"country_name": "employee_residence"})
final_df.head()